In [1]:
from pathlib import Path
from tab_hero.dataio.tab_format import load_tab
import pandas as pd

In [2]:
dir_root = Path().cwd().parent.parent
dir_preprocessed_data = dir_root / 'data' / 'processed'

View preprocessed tab data

In [3]:
for i, tab_file in enumerate(dir_preprocessed_data.glob('*.tab')):
    print(tab_file)
    
    tab_data = load_tab(tab_file)
    
    print(tab_data)
    
    if i == 10:
        break

e:\Python Projects\tab-hero\data\processed\00025e292bfacc9b.tab


error: Error -3 while decompressing data: incorrect header check

In [4]:
for i, tab_file in enumerate(dir_preprocessed_data.iterdir()):
    print(tab_file)
    
    if i == 10:
        break

e:\Python Projects\tab-hero\data\processed\.mel_cache
e:\Python Projects\tab-hero\data\processed\.progress_manifest.json
e:\Python Projects\tab-hero\data\processed\00025e292bfacc9b.tab
e:\Python Projects\tab-hero\data\processed\0003cad26b028e39.tab
e:\Python Projects\tab-hero\data\processed\00044d42d97be12d.tab
e:\Python Projects\tab-hero\data\processed\0005aa02866dddf6.tab
e:\Python Projects\tab-hero\data\processed\0005cc1d3cbc5253.tab
e:\Python Projects\tab-hero\data\processed\00068c50bf7a2a95.tab
e:\Python Projects\tab-hero\data\processed\0006ec38aac643a5.tab
e:\Python Projects\tab-hero\data\processed\000964b8f367c35e.tab
e:\Python Projects\tab-hero\data\processed\000bd4b91be58246.tab
